<a href="https://colab.research.google.com/github/juanfariasdev/Web_Scrapping_cafe/blob/main/02_Web_Scrapping_cafe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Bibliotecas

*   A biblioteca [`requests`](https://pypi.org/project/requests/) do Python é usada para fazer requisições HTTP, permitindo conectar a servidores web, enviar dados e receber respostas;
*   A biblioteca [`pytz`](https://pypi.org/project/pytz/) do Python é usada para trabalhar com fusos horários, permitindo manipulação de datas e horas em diferentes regiões do mundo
*   A biblioteca [`BeautifulSoup`](https://pypi.org/project/beautifulsoup4) do Python é usada para extrair dados de documentos HTML/XML, permitindo navegar, pesquisar e modificar o conteúdo do documento, sendo amplamente utilizada para web scraping.

In [ ]:
import random
import os
import requests
import pytz
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from enum import Enum
from bs4 import BeautifulSoup
from datetime import datetime

### Funções utilitárias

Essas funções serão utilizadas em diferentes partes do notebook.

In [ ]:
# Lista de agentes de usuário
agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:109.0) Gecko/20100101 Firefox/116.0",
    "Mozilla/5.0 (Linux; Android 10; K) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.6 Safari/605.1.15",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36 Edg/115.0.1901.188",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36 Edg/117.0.0.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"
]

# Seleciona um agente de usuário aleatório
def get_agent():
    return random.choice(agents)

# Enum para fontes de dados
class Sources(str, Enum):
    B3 = 'b3'  # Bolsa Brasileira (Brasil)
    CEPEA = 'cepea'  # Cpea/Esalq para SP e interior
    COOPAMA = 'coopama'  # Coopama, Machado MG
    COOPERCAM = 'coopercam'  # Coopercam, Campos Gerais MG
    CCMG = 'ccmg'  # Centro do Comércio de Café, Varginha MG
    COOXUPE = 'cooxupe'  # Cooxupé, Guaxupé MG
    CAFEPOCOS = 'cafe-pocos'  # Café Poços, Poços de Caldas MG
    COCAPEC = 'cocapec'  # Cocapec, Franca SP

    def __str__(self) -> str:
        return self.value

# Obtém a data e hora formatadas
def date_time(separator_date='/', separator_time=':'):
    snap = datetime.now(pytz.timezone('America/Sao_Paulo'))
    return {
        'date': snap.strftime(f"%d{separator_date}%m{separator_date}%Y"),
        'time': snap.strftime(f"%H{separator_time}%M{separator_time}%S")
    }

# Mantém apenas os números (e caracteres extras opcionais) em uma string
def retain_numbers_in_string(str_value, extra_chars=""):
    allowed_chars = "0123456789" + extra_chars
    return ''.join(filter(lambda x: x in allowed_chars, str_value))

# Converte vírgulas em pontos em uma string
def comma_to_point(str_value):
    return str_value.replace(',', '.')


### Variáveis globais

Essas variáveis serão utilizadas em diferentes partes do notebook.

In [ ]:
headers = {'User-Agent': get_agent()}
digits_for_floats = 2

In [ ]:
def fetch_soup(url, headers):
    """ Obtém e parseia o conteúdo HTML da página. """
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Levanta erro para status HTTP ruins
        return BeautifulSoup(response.content, 'html.parser')
    except requests.RequestException as error:
        display(f"Erro ao acessar {url}: {error}")
        return None

### Cotações

Buscaremos cotações de preço de café em diferentes regioões/cidades do Brasil.

A idéia é consolidar essas cotações ao longo do tempo para fazer análises.

#### Cotação na B3

A Brasil, Bolsa, Balcão ([B3](https://www.b3.com.br)) é a bolsa de valores brasileira, que fica sediada na cidade de São Paulo.

In [ ]:
def price_b3_via_coox(url, headers):
    us_coffee = us_dollar = 0

    soup = fetch_soup(url, headers)
    if not soup:
        return 0

    table = soup.find('tbody')
    if not table:
        display('Erro ao buscar tabela de preços B3')
        return 0

    for row in table.find_all('tr'):
        columns = row.find_all('td')

        if columns:
            desc = columns[0].text.strip()
            value = columns[1].text.strip()[:-1]  # Remove o último caractere

            if 'KCZ4' in desc:
                us_coffee = value
            elif 'DOL COM' in desc:
                us_dollar = value

    if not us_coffee or not us_dollar:
        display('Erro ao buscar preços B3')
        return 0

    return round(float(us_coffee) * float(us_dollar), digits_for_floats)

#### Cotação do CPEA

O [CEPEA](https://www.cepea.esalq.usp.br/br) é o Centro de Estudos Avançados em Economia Aplicada da Escola Superior de Agricultura (Esalq/USP).

In [ ]:
def price_cepea(url, headers):
    soup = fetch_soup(url, headers)
    if not soup:
        return 0

    tbody = soup.find('tbody')
    if not tbody:
        display('Erro ao buscar tabela de preços CEPEA')
        return 0

    for row in tbody.find_all('tr'):
        columns = row.find_all('td')
        if columns:
            price = comma_to_point(retain_numbers_in_string(columns[1].text))
            return round(float(price), digits_for_floats)

    display('Erro ao ler os dados do CEPEA')
    return 0

#### Cotação da Coopama

A [Coopama](https://www.coopama.com.br/) é a Cooperativa Agrária de Machado que, entre outras atividades, oferece comercialização e armazenagem de café e outros grãos, como milho, soja, sorgo e aveia.

In [ ]:
def price_coopama(url, headers):
    soup = fetch_soup(url, headers)
    if not soup:
        return 0

    div = soup.find('div', {'id': "cotacao-cafe"})
    if not div:
        display('Erro ao buscar seção de cotação do Coopama')
        return 0

    spans = div.find_all('span', {'class': 'col'})
    found = False

    for span in spans:
        content = span.text.strip()

        if found:
            price = comma_to_point(retain_numbers_in_string(content))
            return round(float(price), digits_for_floats)

        if 'Dura' in content:
            found = True

    display('Erro ao ler os dados do Coopama')

#### Cotação da Coopercam

A [Coopercam](https://coopercam.com.br/) é a Cooperativa do Cafeicultores de Campos Gerais MG e Campo do Meio MG. Sediado no segundo maior município produtor de café do Brasil, a cooperativa atua em aproximadamente 15 mil ha de café, que abrange os munícipios de Campos Gerais, Campo do Meio, distrito de Córrego do Ouro e cidades circunvizinhas.

In [ ]:
def price_coopercam(url, headers):
    soup = fetch_soup(url, headers)
    if not soup:
        return 0

    div = soup.find('div', {'id': "cotacao-cafe"})
    if not div:
        display('Erro ao buscar seção de cotação do Coopama')
        return 0

    spans = div.find_all('span', {'class': 'col'})
    found = False

    for span in spans:
        content = span.text.strip()

        if found:
            price = comma_to_point(retain_numbers_in_string(content))
            return round(float(price), digits_for_floats)

        if 'Dura' in content:
            found = True

    display('Erro ao ler os dados do Coopama')

#### Cotação do CCMG

O [CCMG](https://cccmg.com.br/) é o Centro do Comércio de Café do Estado de Minas Gerais, uma instituição representativa, sem fins lucrativos, que agrega empresas que atuam no agronegócio café.

In [ ]:
def price_ccmg(url, headers):
    soup = fetch_soup(url, headers)
    if not soup:
        return 0

    div = soup.find('div', {'id': "cotacao-cafe"})
    if not div:
        display('Erro ao buscar seção de cotação do Coopama')
        return 0

    spans = div.find_all('span', {'class': 'col'})
    found = False

    for span in spans:
        content = span.text.strip()

        if found:
            price = comma_to_point(retain_numbers_in_string(content))
            return round(float(price), digits_for_floats)

        if 'Dura' in content:
            found = True

    display('Erro ao ler os dados do Coopama')

#### Cotação da Cooxupé

A [Cooxupé](https://www.cooxupe.com.br/) é a Cooperativa Regional de Cafeicultores em Guaxupé MG. Atualmente, é a maior cooperativa de cafeicultores do mundo, contando com mais de 20 mil cooperados espalhados em mais de 330 municípios localizados no Sul de Minas, Cerrado Mineiro, Matas de Minas e Vale do Rio Pardo (no estado de São Paulo).

Ler cotações no [Hub do Café](https://hubdocafe.cooxupe.com.br/cafe-da-tarde/), card **Café fino Cooxupé**.

In [ ]:
def price_cooxupe(url, headers):
    soup = fetch_soup(url, headers)
    if not soup:
        return 0

    div = soup.find('div', {'id': "cotacao-cafe"})
    if not div:
        display('Erro ao buscar seção de cotação do Coopama')
        return 0

    spans = div.find_all('span', {'class': 'col'})
    found = False

    for span in spans:
        content = span.text.strip()

        if found:
            price = comma_to_point(retain_numbers_in_string(content))
            return round(float(price), digits_for_floats)

        if 'Dura' in content:
            found = True

    display('Erro ao ler os dados do Coopama')

#### Cotação da Café Poços

A [Café Poços]() é a Cooperativa Regional dos Cafeicultores de Poços de Caldas.

Ler cotações no [Notícias Agrícolas](https://www.noticiasagricolas.com.br/cotacoes/cafe/cafe-arabica-mercado-fisico-tipo-6-7).

In [ ]:
def price_pocos(url, headers):
    soup = fetch_soup(url, headers)
    if not soup:
        return 0

    div = soup.find('div', {'id': "cotacao-cafe"})
    if not div:
        display('Erro ao buscar seção de cotação do Coopama')
        return 0

    spans = div.find_all('span', {'class': 'col'})
    found = False

    for span in spans:
        content = span.text.strip()

        if found:
            price = comma_to_point(retain_numbers_in_string(content))
            return round(float(price), digits_for_floats)

        if 'Dura' in content:
            found = True

    display('Erro ao ler os dados do Coopama')

In [ ]:
def price_cocapec(url, headers):
    """Obtém o preço de Franca na tabela de cotações da Cocapec."""
    soup = fetch_soup(url, headers)
    if not soup:
        return None

    table = soup.find('table', {'class': 'cot-fisicas'})
    if not table:
        display("Erro: Não foi possível encontrar a tabela de cotações da Cocapec.")
        return None

    for row in table.find_all('tr'):
        columns = row.find_all('td')
        if columns and 'Franca' in columns[0].text:
            price = comma_to_point(retain_numbers_in_string(columns[1].text.strip()))
            return round(float(price), 2)

    display("Erro: Não foi possível encontrar o preço de Franca.")
    return None


### Leitura dos Dados

In [ ]:
# Definindo um dicionário de fontes e URLs
sources_urls = {
    Sources.B3: "https://hubdocafe.cooxupe.com.br",
    Sources.CEPEA: 'https://www.cepea.esalq.usp.br/br/indicador/cafe.aspx',
    Sources.COOPAMA: 'https://www.coopama.com.br',
    Sources.COOPERCAM: 'https://coopercam.com.br/cotacoes/',
    Sources.CCMG: 'https://cccmg.com.br/cotacao-do-cafe/19-12-2024/',
    Sources.COOXUPE: 'https://hubdocafe.cooxupe.com.br/cafe-da-tarde/',
    Sources.CAFEPOCOS: 'https://www.noticiasagricolas.com.br/cotacoes/cafe/cafe-arabica-mercado-fisico-tipo-6-7',
    Sources.COCAPEC: 'https://www.noticiasagricolas.com.br/cotacoes/cafe/cafe-arabica-mercado-fisico-tipo-6-7'
}

# Definindo um dicionário de preços
prices = {
    'data': date_time()['date'],
    Sources.B3: [],
    Sources.CEPEA: [],
    Sources.COOPAMA: [],
    # Sources.COOPERCAM: [],
    # Sources.CCMG: [],
    # Sources.COOXUPE: [],
    # Sources.CAFEPOCOS: [],
    Sources.COCAPEC: []
}

# Função auxiliar para buscar e adicionar preços ao dicionário
def fetch_price(source, url, headers):
    try:
        # Funções de preço específicas para cada fonte
        if source == Sources.B3:
            return price_b3_via_coox(url, headers)
        elif source == Sources.CEPEA:
            return price_cepea(url, headers)
        elif source == Sources.COOPAMA:
            return price_coopama(url, headers)
        # elif source == Sources.COOPERCAM:
        #     return price_coopercam(url, headers)
        # elif source == Sources.CCMG:
        #     return price_ccmg(url, headers)
        # elif source == Sources.COOXUPE:
        #     return price_cooxupe(url, headers)
        # elif source == Sources.CAFEPOCOS:
        #     return price_pocos(url, headers)
        elif source == Sources.COCAPEC:
            return price_cocapec(url, headers)
    except Exception as e:
        display(f"Erro ao buscar dados de {source}: {e}")
        return None

# Iniciando a leitura
display('Leitura iniciada...')

# Iterando sobre as fontes e URLs
for source, url in sources_urls.items():
    price = fetch_price(source, url, headers)
    if price is not None:
        prices[source].append(price)

display('Leitura de dados ok!')


### Salvamento dos Dados

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

# O caminho vai depender do local do arquivo no seu drive
file_name = '/content/drive/MyDrive/precos.csv'

df = pd.DataFrame(prices)

# Testar se o arquivo existe para abrir em edição
if os.path.isfile(file_name):
  df.to_csv(file_name, mode='a', index=False, header=False)
  display("Adicionando informações ao arquivo.")
# Se não existir, abre em criação
else:
  df.to_csv(file_name, index=False)
  display(f"Arquivo '{file_name}' criado com sucesso.")
  display(f"Adicionando informações ao arquivo criado.")

### Visualização dos Dados

In [ ]:
df = pd.read_csv(file_name)
display(df.head())

dfm = df.melt('data', var_name='colunas', value_name='valores')
display(dfm)

#### Gráfico de Evolução de Preços

Gráfico de Linhas.

In [ ]:
sns.set(font_scale=1)

plt.figure(figsize=(10, 6))

graph = sns.lineplot(data=dfm, x='data', y='valores', hue='colunas', marker='o')
graph.set_ylim(260000, 280000)

legends, _ = graph.get_legend_handles_labels()
graph.legend(legends, list(map(lambda leg :
        leg.get_label().capitalize(), legends)), title="Entidades")

graph.set(title="Preços do Café Sul MG", xlabel='Data',
          ylabel="Valors em R$")

plt.show()

### Gráfico de (*sua escolha*)

Explicar qual o gráfico utilizado, e a o que ele pretende mostrar em relação aos dados.

In [ ]:
# Ajustando o estilo global
sns.set(style="whitegrid", font_scale=1.2)

# Definindo o tamanho da figura
plt.figure(figsize=(14, 7))

# Criação do gráfico de barras agrupadas
graph = sns.barplot(data=dfm, x='data', y='valores', hue='colunas', palette="muted", errorbar=None)

# Ajuste de títulos e rótulos com mais estilo
graph.set(title="Preços do Café Sul MG", xlabel='Data', ylabel="Valores em R$")

# Personalizando os rótulos do eixo X para melhorar a legibilidade
plt.xticks(rotation=45, ha='right', fontsize=12)

# Exibindo as barras com bordas finas para uma aparência mais discreta
for container in graph.containers:
    graph.bar_label(container, padding=5, fontsize=10, color='black')

# Adicionando linhas de grade mais discretas
plt.grid(True, linestyle='--', alpha=0.7)

# Ajustando o layout
plt.tight_layout()

# Exibindo o gráfico
plt.show()
